In [1]:
from pathlib import Path

from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from trulens_eval import Tru, FeedbackMode
from trulens_eval.tru_agent import TruAgent, OpenAiFunctionsEventExtractor

In [2]:
# Prepare an empty database file
file = Path("db/tru_agent.sqlite")
file.parent.mkdir(exist_ok=True, parents=True)
file.unlink(missing_ok=True)

In [3]:
tru = Tru(database_url=f"sqlite:///{file}")

In [9]:
tru.start_dashboard(force=True)

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.178.30:8501 .


In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [6]:
app = TruAgent(
    app=agent,
    app_id="OpenAiFunctions",
    events_extractor=OpenAiFunctionsEventExtractor(),
)

_, rec = app.call_with_record(inputs={"input": "what is the square root of 49"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(49)`




> Entering new LLMMathChain chain...
sqrt(49)```text
sqrt(49)
```
...numexpr.evaluate("sqrt(49)")...

Answer: 7.0
> Finished chain.
Answer: 7.0The square root of 49 is 7.

> Finished chain.


In [7]:
df = tru.db.get_events(rec.record_id)
df

,idx,ts,category,content
0,1,2023-08-09 01:15:34.084704,Input,what is the square root of 49
1,2,2023-08-09 01:15:34.084987,Action[Calculator],sqrt(49)
2,3,2023-08-09 01:15:34.085088,Observation,Answer: 7.0
3,4,2023-08-09 01:15:34.085174,AgentFinish,The square root of 49 is 7.
